## Wrangling Milan OpenStreetMap with SQL project

In this project I will use the data wrangling techniques to:
- Audit map data of the city of Milan (Italy)
- Fix problems encountered and enhance data quality
- Import data into an sqlite database to perform some exploratory queries

Where I found the data Map?

I downloaded Milan Map from [Mapzen - Metro Extracts](https://mapzen.com/data/metro-extracts/)
I have chosen [Milan](https://en.wikipedia.org/wiki/Milan) because I have been there several times. Milan is the economic city of Italy because is the main city for business, and Rome is for tourism.
The overall filesize of the unzipped xml file is ~703mb.
After downloading the xml file the data wrangling process I started looking at:
- Element adress street
- Element building
- Element brand    
I found some validity, accuracy and consistency problems in these elements. For example: 
tag k="building" v="garage" />   
tag k="building" v="garages" />  
tag k="building" v="garges" />
tag k="brand" v="Total Erg" />  
tag k="brand" v="TotalErg" />  
tag k="addr:street" v="SP 121 Pobbiano - Agrate" />   
tag k="addr:street" v="SP 121 Pobbiano - Agrate" />   
tag k="addr:street" v="Strada Provinciale 121" />   

The streets in Italy are person's name so a number is not valid, I found two numbers and I changed into the name.
The building have some errors like "garges" should be "garage".
I only used the sample file on these code as my computer run out the memory using the all data.


In [ ]:
### I firsly take a sample from the original using this code
"""
import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "milan_italy.osm"  
SAMPLE_FILE = "Milano.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
      Yield element if it is the right type of tag


    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')
"""

In [2]:
### Import all the libraries that I will use
import os
import xml.etree.ElementTree as ET
from collections import defaultdict
import re
import pprint
import csv
import codecs
import cerberus

In [3]:
os.getcwd()
path = 'C:\Users\SAMSUNG\Documents\Udacity\Nano_degree_Data_analyst\P3 - Wrangle OpenStreetMap Data'
print 'The size in bytes of the data sample is = {} bytes'.format(os.path.getsize('Milano.osm'))
print 'The size in bytes of the original data is = {} bytes'.format(os.path.getsize('Milan_italy.osm'))
print 'The size in bytes of the original data unzziped is = {} bytes'.format(os.path.getsize('milan_italy.osm.bz2'))

The size in bytes of the data sample is = 74581623 bytes
The size in bytes of the original data is = 737747411 bytes
The size in bytes of the original data unziped is = 56250793 bytes


In [5]:
###ITERATIVE PARSING ON THE SAMPLE FILE
"""
I used the iterative parsing to to process the map file and find out not only what tags and k attributes are there, but also how many, to get the
also how much of which data you can expect to have in the map.
"""

def count_tags(filename):
    
    with open(filename, 'rt') as f:
        tree = ET.parse(filename)
        
        count = dict()
        for child in tree.iter():
            if child.tag in count:
                count[child.tag] = count[child.tag] + 1
            else:   
                count[child.tag] = 1
    return count

def count_keys(filename):
            
    keys = dict()
    for event, elem in ET.iterparse(filename, events = ('start',)):
        if elem.tag == 'way':
            for tag in elem.iter('tag'):
                k = tag.attrib['k']
                if k in keys: 
                    keys[k] = keys[k] + 1
                else:
                    keys[k] = 1
                        
    return keys

def test():

    tags = count_tags('Milano.osm')
    keys = count_keys('Milano.osm')
    print 'Tags type and number'
    pprint.pprint(tags)
    print 'keys type and number'
    pprint.pprint(keys)
    

if __name__ == "__main__":
    test()

Tags type and number
{'member': 10568,
 'nd': 402843,
 'node': 330438,
 'osm': 1,
 'relation': 1266,
 'tag': 169660,
 'way': 49979}
keys type and number
{'abandoned': 5,
 'abandoned:building': 1,
 'abandoned:highway': 1,
 'abutters': 1,
 'access': 1408,
 'access:backward': 2,
 'access:conditional': 8,
 'access:forward': 4,
 'access:lanes': 1,
 'addr:city': 652,
 'addr:country': 320,
 'addr:housename': 35,
 'addr:housenumber': 479,
 'addr:inclusion': 1,
 'addr:interpolation': 30,
 'addr:place': 9,
 'addr:postcode': 619,
 'addr:province': 129,
 'addr:street': 711,
 'admin_level': 80,
 'aerialway': 1,
 'aeroway': 21,
 'agricultural': 12,
 'alt_name': 260,
 'alt_name:2': 13,
 'alt_ref': 23,
 'amenity': 1636,
 'animal': 1,
 'area': 217,
 'artist_name': 1,
 'artwork_type': 1,
 'asphalt': 1,
 'atm': 3,
 'author': 1,
 'automated': 1,
 'backrest': 11,
 'barrier': 1662,
 'basin': 2,
 'bench': 5,
 'bicycle': 882,
 'bicycle:backward': 4,
 'bicycle:forward': 3,
 'bicycle_parking': 2,
 'bicycle_road

In [6]:
### TAGS TYPE

"""
Here we can have the count of each of four tag categories in a dictionary to check the
"k" value for each "<tag>" and see if there are any potential problems.:
  "lower", for tags that contain only lowercase letters and are valid,
  "lower_colon", for otherwise valid tags with a colon in their names,
  "problemchars", for tags with problematic characters, and
  "other", for other tags that do not fall into the other three categories.
See the 'process_map' and 'test' functions for examples of the expected format.
"""


lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        k = element.attrib['k']
        if re.search(lower, k):
            keys["lower"] = keys["lower"] + 1
        elif re.search(lower_colon, k):
            keys["lower_colon"] = keys["lower_colon"] + 1
        elif re.search(problemchars, k):
            keys["problemchars"] = keys["problemchars"] + 1
        else:
             keys["other"] = keys["other"] + 1
        pass
        
    return keys



def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys



def test():
    # You can use another testfile 'map.osm' to look at your solution
    # Note that the assertion below will be incorrect then.
    # Note as well that the test function here is only used in the Test Run;
    # when you submit, your code will be checked against a different dataset.
    keys = process_map('Milano.osm')
    pprint.pprint(keys)


if __name__ == "__main__":
    test()

{'lower': 143750, 'lower_colon': 23813, 'other': 2097, 'problemchars': 0}


In [7]:
### EXPLORING USERS

"""
Find out how many unique users
have contributed to the map in this particular area!

The function process_map should return a set of unique user IDs ("uid")
"""

def get_user(element):
    return


def process_map(filename):
    users = set()
    for user, element in ET.iterparse(filename):
        if element.tag == "node":
           users.add(element.attrib['uid'])
        elif element.tag == "way": 
              users.add(element.attrib['uid'])
        elif element.tag == "relation": 
             users.add(element.attrib['uid'])    
        pass

    return users


def test():

    users = process_map('Milano.osm')
    pprint.pprint(len(users))
    
if __name__ == "__main__":
    test()

1373


In [8]:
### EXPLORING STREET NAMES USING SAMPLE FILE

"""
Find out how many street we have
"""

osm_file = open("Milano.osm", "r")

street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
street_types = defaultdict(int)

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()

        street_types[street_type] += 1

def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print "%s: %d" % (k, v) 

def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street") 
    
    
def audit():
    for event, elem in ET.iterparse(osm_file):
        if is_street_name(elem):
            audit_street_type(street_types, elem.attrib['v'])    
    print_sorted_dict(street_types)    

if __name__ == '__main__':
    audit()

121: 1
23°: 1
Abba: 2
Abbadesse: 1
Abruzzi: 4
Acacie: 1
Acqualunga: 4
Adamello: 7
Adda: 9
Addetta: 1
Adige: 8
Adria: 1
Adua: 4
Affori: 1
Africa: 1
Agadir: 1
Agnesi: 1
Agostoni: 1
Agrate: 2
Agreste: 1
Aguilhon: 1
Aicardo: 1
Aires: 4
Airolda: 1
Airolo: 1
Alba: 1
Albertis: 1
Alberto: 7
Aldina: 5
Aleardi: 1
Alemanni: 2
Alessandrina: 1
Alfieri: 12
Alfonsine: 2
Alighieri: 31
Aliprandi: 4
Allegranza: 1
Allegri: 1
Allende: 1
Alpi: 1
Alpine: 1
Alpini: 4
Alserio: 3
Amadeo: 2
Amati: 5
Amedeo: 1
Amendola: 7
Amiata: 1
Amicis: 8
Ampère: 6
Andegari: 2
Andreani: 1
Andreoli: 1
Angelico: 2
Annone: 1
Annoni: 5
Ansaloni: 1
Ansperto: 1
Antico: 10
Antonini: 1
Aosta: 2
Aporti: 1
Appennini: 2
Appiani: 5
Aprica: 1
aprile: 1
Aprile: 13
Aquileia: 2
Arbe: 5
Archimede: 5
Ardigò: 2
Arese: 2
Arezzo: 1
Argelati: 1
Argonne: 1
Ariosto: 6
Aristotele: 6
Arno: 6
Arona: 1
Arosio: 1
Arsizio: 1
artigiani: 1
Arzaga: 1
Aselli: 2
Asiago: 4
Aspromonte: 3
Assunta: 2
Astesani: 3
Asti: 1
Astolfo: 1
Aulenti: 3
Aurispa: 1
Ausonio: 1


In [9]:
### EXPLORING BUILDINGS TYPES USING SAMPLE FILE

"""
Find out how many type of buildings we have
"""

osm_file = open("Milano.osm", "r")

street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
street_types = defaultdict(int)

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()

        street_types[street_type] += 1

def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print "%s: %d" % (k, v) 

def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "building") 
    
    
def audit():
    for event, elem in ET.iterparse(osm_file):
        if is_street_name(elem):
            audit_street_type(street_types, elem.attrib['v'])    
    print_sorted_dict(street_types)    

if __name__ == '__main__':
    audit()

abandoned: 1
apartments: 1986
barn: 5
bridge: 1
castle: 1
chapel: 3
church: 64
civic: 10
collapsed: 7
commercial: 158
construction: 10
cowshed: 2
damaged: 1
depot: 1
detached: 19
farm: 43
farm_auxiliary: 15
fire_station: 1
garage: 206
garages: 168
garges: 1
generic: 1
grandstand: 8
greenhouse: 12
gym: 5
hangar: 6
hospital: 12
hotel: 11
house: 2053
hut: 2
industrial: 944
kindergarten: 11
kiosk: 11
manufacture: 1
military: 1
monastery: 1
office: 69
parish: 8
parking: 1
pavilion: 1
platform: 1
prison: 1
public: 18
residential: 1379
restaurant: 1
retail: 62
Rho: 1
roof: 169
sally_port: 1
school: 89
service: 131
shed: 59
shelter: 4
stable: 2
stadium: 1
supermarket: 3
tank: 3
terrace: 107
tower: 2
train_station: 10
transformer_tower: 1
university: 10
utility: 3
villa: 5
warehouse: 21
wayside_shrine: 2
window: 1
yes: 14100


In [10]:
### EXPLORING BUILDINGS TYPES USING SAMPLE FILE

"""
Find out how many type of buildings we have
"""

osm_file = open("Milano.osm", "r")

street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
street_types = defaultdict(int)

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()

        street_types[street_type] += 1

def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print "%s: %d" % (k, v) 

def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "brand") 
    
    
def audit():
    for event, elem in ET.iterparse(osm_file):
        if is_street_name(elem):
            audit_street_type(street_types, elem.attrib['v'])    
    print_sorted_dict(street_types)    

if __name__ == '__main__':
    audit()

@pump: 1
Agesp: 1
Agip: 8
api: 2
Aprilia: 1
Auchan: 1
Audi: 1
BCC: 1
Beghelli: 1
Bergamasco: 1
BNL: 1
Brums: 1
cioccolato: 1
DR: 1
Easy: 1
Enel: 2
Eni: 7
ERG: 4
Erg: 2
Esso: 7
Farmacia: 1
glass: 1
Honda: 1
Independent: 1
IP: 7
Keropetrol: 1
Lidl: 1
Lillapois: 1
Mazda: 1
Mercedes: 1
Mistubishi: 1
Oil: 1
Opel: 1
Peugeot: 2
Puff: 1
Q8: 6
Q8easy: 2
Retitalia: 1
Romeo: 1
Shell: 11
store: 1
Tamoil: 10
Tecnorete: 1
TIM: 1
Tim: 1
Total: 6
TotalErg: 2
Unes: 1
Vodafone: 1
West: 1
Yamaha: 1


In [11]:
###Improving Street Names, building types and brand on the original file
"""
- audit the OSMFILE and change the variable 'mapping' to reflect the changes needed to fix 
    the unexpected street types to the appropriate ones in the expected list.
    You have to add mappings only for the actual problems you find in this OSMFILE,
    not a generalized solution, since that may and will depend on the particular area you are auditing.
- write the update_name function, to actually fix the street name.
    The function takes a string with street name as an argument and should return the fixed name
    We have provided a simple test so that you see what exactly is expected
"""


OSMFILE = "Milano.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

        
expected = ["Aprile", "Bellini", "XXIII", "121 Pobbiano - Agrate", "garage", "Api", "TotalErg", "Q8easy", "Tim"]

mapping = { "aprile": "Aprile",
            "bellini": "Bellini",
            "Ave" : "",
            "23°" : "XXIII",
            "121" : "121 Pobbiano - Agrate",
            "garages" : "garage",
            "garges" : "garage",
            "api" : "Api",
            "TIM" : "Tim",
            "Erg" : "TotalErg",
            "Easy" : "Q8easy"
            }
    
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street" or elem.attrib['k'] == "building" or elem.attrib['k'] == "brand")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types


def update_name(name, mapping):
    
    n = street_type_re.search(name)
    
    if n.group() in mapping.keys():
        if n not in expected:
            name = re.sub(n.group(), mapping[n.group()], name)
    return name


def test():
    st_types = audit(OSMFILE)
    pprint.pprint(dict(st_types))

    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
            print name, "=>", better_name
            if name == "aprile":
                assert better_name == "Aprile"
            if name == "Erg":
                assert better_name == "TotalErg"


if __name__ == '__main__':
    test()

{'121': set(['Strada Provinciale 121']),
 'Abba': set(['Via Giuseppe Cesare Abba']),
 'Abbadesse': set(['Via Abbadesse']),
 'Abruzzi': set(['Viale Abruzzi']),
 'Acacie': set(['Via delle Acacie']),
 'Acqualunga': set(['Via Acqualunga']),
 'Adamello': set(['Via Adamello']),
 'Adda': set(['Via Adda', 'Via Fiume Adda']),
 'Addetta': set(['Viale Addetta']),
 'Adige': set(['Via Adige']),
 'Adria': set(['Via Adria']),
 'Adua': set(['Via Adua']),
 'Affori': set(['Viale Affori']),
 'Africa': set(['Via Africa']),
 'Agadir': set(['Via Agadir']),
 'Agesp': set(['Agesp']),
 'Agip': set(['Agip']),
 'Agnesi': set(['Via Gaetana Agnesi']),
 'Agostoni': set(['Via Angelo Agostoni']),
 'Agrate': set(['SP 121 Pobbiano - Agrate']),
 'Agreste': set(['Via Agreste']),
 'Aguilhon': set(['Via Cesare Aguilhon']),
 'Aicardo': set(['Via Aicardo']),
 'Aires': set(['Corso Buenos Aires', 'Galleria Buenos Aires']),
 'Airolo': set(['Via Airolo']),
 'Alba': set(['Vicolo Alba']),
 'Albertis': set(['Via Sebastiano de Alber

C:\Users\SAMSUNG\Anaconda3\envs\dato-env\lib\site-packages\ipykernel\__main__.py:61: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


Via Enrico Fermi
Via Orti => Via Orti
Via Emilia => Via Emilia
Via Jacopo Sansovino => Via Jacopo Sansovino
Via Sansovino => Via Sansovino
Via Casecca => Via Casecca
Via Sandro Botticelli => Via Sandro Botticelli
Via Carlo Goldoni => Via Carlo Goldoni
Via della Cascina Selva => Via della Cascina Selva
Via Francesco Perotti => Via Francesco Perotti
Via Andrea Maria Ampère => Via Andrea Maria Ampère
Via Giuseppe Mazzini => Via Giuseppe Mazzini
Piazza Giuseppe Mazzini => Piazza Giuseppe Mazzini
Via Caprera => Via Caprera
Via Fratelli Cervi => Via Fratelli Cervi
Via Monte Generoso => Via Monte Generoso
Via Beniamino Gigli => Via Beniamino Gigli
Via Italia => Via Italia
Corso Italia => Corso Italia
Piazza Italia => Piazza Italia
Via Giovanni Enrico Pestalozzi => Via Giovanni Enrico Pestalozzi
Via Eching => Via Eching
farm_auxiliary => farm_auxiliary
Via Raffaele Merelli => Via Raffaele Merelli
Via Giuseppe Cesare Abba => Via Giuseppe Cesare Abba
Via delle Palme => Via delle Palme
Spalto Pio

In [ ]:
###Preparing for Database - SQL(data.py) using the sample file


"""
After auditing is complete the next step is to prepare the data to be inserted into a SQL database.
To do so you will parse the elements in the OSM XML file, transforming them from document format to
tabular format, thus making it possible to write to .csv files.  These csv files can then easily be
imported to a SQL database as tables.

The process for this transformation is as follows:
- Use iterparse to iteratively step through each top level element in the XML
- Shape each element into several data structures using a custom function
- Utilize a schema and validation library to ensure the transformed data is in the correct format
- Write each data structure to the appropriate .csv files

## Shape Element Function
The function should take as input an iterparse Element object and return a dictionary.

### If the element top level tag is "node":
The dictionary returned should have the format {"node": .., "node_tags": ...}

The "node" field should hold a dictionary of the following top level node attributes:
- id
- user
- uid
- version
- lat
- lon
- timestamp
- changeset
All other attributes can be ignored

The "node_tags" field should hold a list of dictionaries, one per secondary tag. Secondary tags are
child tags of node which have the tag name/type: "tag". Each dictionary should have the following
fields from the secondary tag attributes:
- id: the top level node id attribute value
- key: the full tag "k" attribute value if no colon is present or the characters after the colon if one is.
- value: the tag "v" attribute value
- type: either the characters before the colon in the tag "k" value or "regular" if a colon
        is not present.

Additionally,

- if the tag "k" value contains problematic characters, the tag should be ignored
- if the tag "k" value contains a ":" the characters before the ":" should be set as the tag type
  and characters after the ":" should be set as the tag key
- if there are additional ":" in the "k" value they and they should be ignored and kept as part of
  the tag key. For example:

  <tag k="addr:street:name" v="Lincoln"/>
  should be turned into
  {'id': 12345, 'key': 'street:name', 'value': 'Lincoln', 'type': 'addr'}

- If a node has no secondary tags then the "node_tags" field should just contain an empty list.

The final return value for a "node" element should look something like:

{'node': {'id': 757860928,
          'user': 'uboot',
          'uid': 26299,
       'version': '2',
          'lat': 41.9747374,
          'lon': -87.6920102,
          'timestamp': '2010-07-22T16:16:51Z',
      'changeset': 5288876},
 'node_tags': [{'id': 757860928,
                'key': 'amenity',
                'value': 'fast_food',
                'type': 'regular'},
               {'id': 757860928,
                'key': 'cuisine',
                'value': 'sausage',
                'type': 'regular'},
               {'id': 757860928,
                'key': 'name',
                'value': "Shelly's Tasty Freeze",
                'type': 'regular'}]}

### If the element top level tag is "way":
The dictionary should have the format {"way": ..., "way_tags": ..., "way_nodes": ...}

The "way" field should hold a dictionary of the following top level way attributes:
- id
-  user
- uid
- version
- timestamp
- changeset

All other attributes can be ignored

The "way_tags" field should again hold a list of dictionaries, following the exact same rules as
for "node_tags".

Additionally, the dictionary should have a field "way_nodes". "way_nodes" should hold a list of
dictionaries, one for each nd child tag.  Each dictionary should have the fields:
- id: the top level element (way) id
- node_id: the ref attribute value of the nd tag
- position: the index starting at 0 of the nd tag i.e. what order the nd tag appears within
            the way element

The final return value for a "way" element should look something like:

{'way': {'id': 209809850,
         'user': 'chicago-buildings',
         'uid': 674454,
         'version': '1',
         'timestamp': '2013-03-13T15:58:04Z',
         'changeset': 15353317},
 'way_nodes': [{'id': 209809850, 'node_id': 2199822281, 'position': 0},
               {'id': 209809850, 'node_id': 2199822390, 'position': 1},
               {'id': 209809850, 'node_id': 2199822392, 'position': 2},
               {'id': 209809850, 'node_id': 2199822369, 'position': 3},
               {'id': 209809850, 'node_id': 2199822370, 'position': 4},
               {'id': 209809850, 'node_id': 2199822284, 'position': 5},
               {'id': 209809850, 'node_id': 2199822281, 'position': 6}],
 'way_tags': [{'id': 209809850,
               'key': 'housenumber',
               'type': 'addr',
               'value': '1412'},
              {'id': 209809850,
               'key': 'street',
               'type': 'addr',
               'value': 'West Lexington St.'},
              {'id': 209809850,
               'key': 'street:name',
               'type': 'addr',
               'value': 'Lexington'},
              {'id': '209809850',
               'key': 'street:prefix',
               'type': 'addr',
               'value': 'West'},
              {'id': 209809850,
               'key': 'street:type',
               'type': 'addr',
               'value': 'Street'},
              {'id': 209809850,
               'key': 'building',
               'type': 'regular',
               'value': 'yes'},
              {'id': 209809850,
               'key': 'levels',
               'type': 'building',
               'value': '1'},
              {'id': 209809850,
               'key': 'building_id',
               'type': 'chicago',
               'value': '366409'}]}


import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

OSM_PATH = "Milano.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def correct_k(k):
    index=k.find(':')
    typ=k[:index]
    k=k[index+1:]    
    return k,typ

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # YOUR CODE HERE
    if element.tag=='node':
        for i in node_attr_fields:
            node_attribs[i]=element.attrib[i]
            
    if element.tag=='way':
        for i in way_attr_fields:
            way_attribs[i]=element.attrib[i]
        
    for tag in element.iter("tag"):
        dic={}
        attributes=tag.attrib
        if problem_chars.search(tag.attrib['k']):
            continue
        
        if element.tag=='node':
            dic['id']=node_attribs['id']
        else:
            dic['id']=way_attribs['id']
        
        dic['value'] = attributes['v']

        colon_k=LOWER_COLON.search(tag.attrib['k'])
        if colon_k:
            print colon_k.group(0)
            print tag.attrib['k']
            dic['key'],dic['type']=correct_k(tag.attrib['k'])
        else:
            dic['key']=attributes['k']
            dic['type']='regular'
        
        
        #print dic
        tags.append(dic)
    
    if element.tag=='way':
        position=0
        for nd in element.iter("nd"):
            way_node_dic={}
            way_node_dic['id']=way_attribs['id']
            way_node_dic['node_id']=nd.attrib['ref']
            way_node_dic['position']=position
            position = position + 1
            way_nodes.append(way_node_dic)
    
    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}
     


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    Yield element if it is the right type of tag

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    Raise ValidationError if element does not match schema
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    Extend csv.DictWriter to handle Unicode input

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    Iteratively process each XML element and write to csv(s)

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)
"""   

In [3]:
path = 'C:\Users\SAMSUNG\Documents\Udacity\Nano_degree_Data_analyst\P3 - Wrangle OpenStreetMap Data'
print 'The size in bytes of nodes = {} bytes'.format(os.path.getsize('nodes.csv'))
print 'The size in bytes of nodes tags = {} bytes'.format(os.path.getsize('nodes_tags.csv'))
print 'The size in bytes of ways = {} bytes'.format(os.path.getsize('ways.csv'))
print 'The size in bytes of ways node is = {} bytes'.format(os.path.getsize('ways_nodes.csv'))
print 'The size in bytes of ways tags is = {} bytes'.format(os.path.getsize('ways_tags.csv'))

The size in bytes of nodes = 26908052 bytes
The size in bytes of nodes tags = 2170623 bytes
The size in bytes of ways = 2984762 bytes
The size in bytes of ways node is = 10049963 bytes
The size in bytes of ways tags is = 4205980 bytes


In [ ]:
##### Importing the data on sql
###  sqlite3 MilanMap.db
###     .mode csv
###    .import nodes.csv nodes
###    .import ways.csv ways
###    .import nodes_tags.csv nodes_tags
###    delete from nodes_tags where id = 'id';
###    .import ways_tags.csv ways_tags
###    delete from ways_tags where id = 'id';
###    .import ways_nodes.csv ways_nodes
###    delete from ways_nodes where id = 'id';

In [ ]:
### Quering the database with SQL   
"""
Number of nodes    
SELECT COUNT(*) FROM nodes;    
330438     

Number of ways    
SELECT COUNT(*) FROM ways;    
49979    

Top 5 contributing users    
SELECT t.user, COUNT(*) as Total_users    
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) t    
GROUP BY t.user       
ORDER BY Total_users DESC    
LIMIT 5;      

Alecs01,52948     
ilrobi,31997
adirricor,25865
Guido_RL,23109
fedc,15701      

Top 5 most common name of street in Milan     
SELECT tags.value as Street_Name, COUNT(*) as Total_count   
FROM (SELECT * FROM nodes_tags UNION ALL    
      SELECT * FROM ways_tags) tags     
WHERE tags.key = 'street'    
GROUP BY Street_Name    
ORDER BY Total_count DESC    
LIMIT 5;    

"Via Giuseppe Garibaldi",40      
"Via Alessandro Manzoni",35     
"Via Alessandro Volta",33     
"Via Roma",33    
"Via Piave",32     

Top 5 type of buildings in Milan     
SELECT tags.value as Building_Type, COUNT(*) as Total_building     
FROM (SELECT * FROM nodes_tags UNION ALL        
      SELECT * FROM ways_tags) tags      
WHERE key ='building' and value != 'yes'     
GROUP BY Building_Type    
ORDER BY Total_building DESC    
LIMIT 5;    
house,2050     
apartments,1974     
residential,1372     
industrial,937     
garage,206     

Top 5 shop type in Milan     
SELECT tags.value as Shop_Type, COUNT(*) as Total_shop      
FROM (SELECT * FROM nodes_tags UNION ALL     
      SELECT * FROM ways_tags) tags   
WHERE key ='shop'
GROUP BY Shop_Type
ORDER BY Total_shop DESC
LIMIT 5;
clothes,106
supermarket,68
hairdresser,58
bakery,37
car_repair,33

Top 5 of cuisine in Milan
SELECT nodes_tags.value as Restaurant_Type, COUNT(*) as Total_count
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') i
    ON nodes_tags.id=i.id
WHERE nodes_tags.key='cuisine'
GROUP BY Restaurant_Type
ORDER BY Total_count DESC
LIMIT 5;
pizza,33
italian,26
regional,8
sushi,5
chinese,4

For curiosity I also wanted to find the top 5 brand name we can have in Milan.
SELECT tags.value as Brand_Name, COUNT(*) as Total_count 
FROM (SELECT * FROM nodes_tags UNION ALL 
      SELECT * FROM ways_tags) tags
WHERE tags.key = 'brand'
GROUP BY Brand_Name
ORDER BY Total_count DESC
LIMIT 5;
Shell,11
Tamoil,10
Agip,8
Eni,7
Esso,7
"""

Using this database we can see that the most common name for street is Giuseppe Garibaldi who was a central figure in the Italian Risorgimento.      
The most type of buidings is house which is unexpected as I always though that there more apartments than buildings in Milan.    
As I expected the most type of shop are for clothes, in fact Milan is know as the "shopping city" and the most common restaurant is pizza as Italy is the country of pizza.   
The most common brand is the petrol brand Shell for fuel.    

### Other ideas about the datasets

A good idea could be to find in which streets are these clothes shops? Which address code had the most common name of Garibaldi?       
Milan is a big city so found these others queries could help to know it better.      
I didn't extract the whole name of streets because I was only interest in the lastname, but I saw that there are some inaccuracies. So this could be an idea to improve the data.       
There is a little gap between the house type and apartments, maybe we can go deeper on this also.       